## Урок 3. Построение модели классификации.

##### 

### 1. Для чего и в каких случаях полезны различные варианты усреднения для метрик качества классификации: micro, macro, weighted?

**Macro-average (для метрик precision, recall, F-score)**

Алгоритм расчета:

Сначала будет посчитана метрика для каждого из классов по отдельности, а затем на основе этих значений будет вычислена итоговая средняя метрика. Т.е. средняя по каждому классу вносит одинаковый вклад в итоговую среднюю.

**Micro-average (для метрик precision, recall, F-score)**

Алгоритм расчета:

Значения TP, TN, FP, FN всех классов будут использованы для расчета итоговой метрики (минуя промежуточный этап, описанный в предыдущем примере). Т.е. итоговая средняя зависит от индивидуального вклада каждого класса.


**Weighted-average (для метрик precision, recall, F-score)**

Алгоритм расчета:

Сначала будет посчитана метрика для каждого из классов по отдельности, а затем на основе этих значений, а также весов/количеств наблюдений конкретного класса будет вычислена итоговая средняя метрика. Т.е. наиболее часто встречаемый класс вносит наибольший вклад в метрику.

***Выводы:***

- В случае мультиклассовой классификации при наличии дисбаланса классов Micro-average лучше отображает реальное качество модели, чем два других усреднения
- Weighted-average может быть полезна, когда мы даем приоритет наиболее представленному классу.
- Macro-average дает грубую приближенную картину.



(Источник: https://datascience.stackexchange.com/questions/15989/micro-average-vs-macro-average-performance-in-a-multiclass-classification-settin)

##### 

### 2. В чём разница между моделями xgboost, lightgbm и catboost или какие их основные особенности?

Все три модели используют методику, известную как градиентный бустинг, для решения задач классификации, регрессии и ranking. Вкратце, идея градиентного бустинга состоит в организации слабых моделей (обычно, но не обязательно, деревьев решений) в ансамбли. В таком ансамбле модели раположены последовательно (а не параллельно, как в бэггинге, например). Каждая последующая модель в ансамбле обучается исправлять ошибки предыдущей модели, т.о. максимизируя некоторую метрику (или минимизируя функцию потерь), например, R2, MSE, RMSE, F-score, Accuracy, Recall и др. При использовании бустинга следует правильно выбирать критерии останова (количество последовательных моделей в ансамбле, гиперпараметры каждой отдельной модели и т.д.) чтобы избежать переобучения.

![Одиночная модель vs Bagging vs Boosting](https://miro.medium.com/max/700/1*PaXJ8HCYE9r2MgiZ32TQ2A.png) 

Упрощенный пример алгоритма бустинга: пусть y - наблюдаемое вещественное значение предсказываемого параметра, а y_пред_1 - предсказанное на первом шаге алгоритма значение, error_1 = y - y_пред_1, тогда error_n = y - у_пред_1 - f(error_2) - ... - f(error_n-1). Последовательно применяя функцию f (из некоторого набора функций) к ошибке с предыдущей итерации, можно минимизировать итоговое значение ошибки.Чтобы перейти от предсказания вещественного значения к предсказанию класса (для задач классификации), используются некоторые пороговые значения, позволяющие отнести вещественный результат к тому или иному классу.

Методика градиентного бустинга является общим случаем для алгоритмов градиентного спуска (используемого в т.ч. в искусственных нейронных сетях).

Теперь рассмотрим каждую из предложенных в задании моделей в отдельности.
** **

**XGBoost**

XGBoost (сокращение от Extreme Gradient Boosting) - впервые опубликованный в 2014г алгоритм, использующий оптимизированную методику градиентного бустинга. 

Особенности алгоритма:
- Использует деревья решений.
- Автоматическая регуляризация данных
- [Стохастический градиентный бустинг](https://www.kaggle.com/pavansanagapati/stochastic-gradient-boosting-with-xgboost) - модели обучаются на подвыборках строк и столбцов исходного набора данных.
- Способность обрабатывать пропуски в данных.
- Возможность дообучения модели на новых данных.

Особенности реализации:
- Параллелизация - использование всех ядер микропроцессора.
- Возможность обучения на GPU
- Распределенные вычисления - при большом объеме данных модель можно обучать на кластере компьютеров.

К минусам алгоритма можно отнести невозможность автоматической [балансировки классов](https://elitedatascience.com/imbalanced-classes).
** **


**LightGBM**

LightGBM (сокращение от Light Gradient Boosting Machine) - впервые опубликованный в 2016г алгоритм от Microsoft, использующий градиентный бустинг.

Особенности алгоритма:
- Использует деревья решений, может также использовать cлучайный лес или [DART деревья](https://arxiv.org/abs/1505.01866).
- Использует подход [GOSS](https://papers.nips.cc/paper/2016/file/9188905e74c28e489b44e954ec0b9bca-Paper.pdf) -   cтроит деревья по принципу "Leaf wise" начиная от листьев с максимальным значением ошибки, что повышает эффективность обучения.
- Использует методику EFB (exclusive feature bundling) - автоматически совмещает несколько признаков в один без потери информации/качества обучения.
- Регуляризация данных
- Возможность дообучения модели на новых данных.


Особенности реализации:
- [Быстрый алгоритм](https://blog.griddynamics.com/xgboost-vs-catboost-vs-lightgbm-which-is-best-for-price-prediction/) (в среднем, быстрее чем XGBoost и CatBoost)
- Экономичный в отношении RAM
- Параллелизация
- Возможность обучения на GPU
** **


**CatBoost**

CatBoost (сокращение от Сategorical Boosting) - впервые опубликованный в 2017г алгоритм от Яндекс, используемый в продуктах компании (поисковик, Яндекс-дзен)

Особенности алгоритма:
- Использует деревья решений (Oblivious trees или Binary Symmetric trees)
- Может автоматически бороться с переобучением (не всегда эффективно)
- В отличие от XGBoost и LightGBM может работать с категориальными признаками (т.е. подготовка данных для модели становится значительно менее трудоемкой)
- В среднем, позволяет достичь чуть более качественных предсказаний/метрик, чем два других рассматриваемых алгоритма.

Особенности реализации:
- В отличие от XGBoost и LightGBM есть визуализация процесса обучения (можно в реальном времени наблюдать итерации обучения и визуально определить, когда его можно можно остановить, если началось переобучение)
- Не очень быстрый, если использовать дефолтные гиперпараметры
- Возможность обучения на GPU
** **

***Резюме:***

Однозначного лидера среди рассмотренных алгоритмов нет. При наличи времени, следует испробовать все три алгоритма и выбрать тот, который показывает лучшие результаты на используемых данных. Если есть желание сэкономить на переводе категориальных признаков в числовые/бинарные, можно использовать CatBoost. Если скорость/производительность обучения очень критична, можно использовать LightGBM (или кластер компьютеров с XGBoost).






